In [1]:
# ac/utils.py
import numpy as np
import pandas as pd
from datetime import datetime
import importlib.util
from pathlib import Path
from arcticdb import Arctic, QueryBuilder

def initialize_db(db_path):
    global ac
    ac_local = Arctic(f'lmdb://{db_path}?map_size=5MB')
    
    if not "general" in ac_local.list_libraries():
        print("Creating library 'general' where *settings and *strategies will be stored")
        library = ac_local.get_library('general', create_if_missing=True)
        index_values = ['port', 
                        's3_db_management', # False for local
                        'aws_access_id', 'aws_access_key',
                        'bucket_name','region',
                        'start_tws','username','password']
        
        data = {'Value': ["7497", # default port
                        "False", # defaul local
                        "", "", # aws_access_id, key
                        "", "", # bucket_name, region
                        "False","","" # Start TWS Automatically default False
                        ]}
        df = pd.DataFrame(data, index=index_values)
        library.write(symbol="settings",data=df)
        ac = ac_local
        return ac
    
    else: # read local settings if settings table exists
        library = ac_local.get_library('general', create_if_missing=True)
        settings_df = library.read("settings").data
        
        # if S3 is set, change ac from local to s3
        if settings_df.loc["s3_db_management","Value"] == str(True):
            region = settings_df.loc["region","Value"]
            bucket_name = settings_df.loc["bucket_name","Value"]
            id = settings_df.loc["aws_access_id","Value"]
            key = settings_df.loc["aws_access_key","Value"]
        
            ac =Arctic(f's3://s3.{region}.amazonaws.com:{bucket_name}?region={region}&access={id}&secret={key}')

            # check if "settings" exists in s3 ac
            if not "general" in ac.list_libraries():
                lib = ac.get_library('general', create_if_missing=True)
                # copy settings from local ac
                lib.write("settings", settings_df)
            elif not "settings" in ac.get_library('general', create_if_missing=True).list_symbols():
                ac.get_library('general', create_if_missing=True).write("settings", settings_df)
        else:
            ac = ac_local
        return ac

In [16]:
ac = Arctic(f'lmdb://db?map_size=5MB')
lib = ac.get_library("general")



In [2]:
# ac = Arctic(f'lmdb://db?map_size=5MB')
ac = initialize_db("db")

In [3]:
ac.list_libraries()

['general', 'settings', 'spx500']

In [4]:
lib = ac.get_library('spx500')

In [5]:
lib.list_symbols()

['ALL_STOCKS']

In [6]:
df = lib.read('ALL_STOCKS').data
df

,Symbol,Open,High,Low,Close,Volume,Name,Sector,Industry,ATR_x,...,ATR,50D_SMA,200D_SMA,1M,3M,6M,12M,RS IBD,RS Rank,RS Rank 20D MA
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-02,A,145.937737,150.436930,145.760129,148.976669,1514100.0,Agilent Technologies,Health Care,Health Care Equipment,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-02,NSC,258.841190,269.713893,257.693693,266.625183,2057300.0,Norfolk Southern Railway,Industrials,Rail Transportation,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-02,ADP,219.492921,224.980491,219.492921,223.375534,2121700.0,ADP,Industrials,Human Resource & Employment Services,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-02,NRG,33.135399,34.163069,33.042814,33.839027,2269300.0,NRG Energy,Utilities,Independent Power Producers & Energy Traders,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-02,NOW,599.570007,625.799988,598.640015,623.369995,1460000.0,ServiceNow,Information Technology,Systems Software,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,K,52.470001,52.910000,52.040001,52.880001,1957400.0,Kellanova,Consumer Staples,Packaged Foods & Meats,0.905175,...,0.905175,51.372696,58.094249,0.056512,-0.049351,-0.146311,-0.196364,-0.441376,0.122490,0.136847
2023-12-01,KDP,31.660000,32.189999,31.629999,32.169998,8212900.0,Keurig Dr Pepper,Consumer Staples,Soft Drinks & Non-alcoholic Beverages,0.523807,...,0.523807,30.710227,32.412652,0.056139,-0.034579,0.036594,-0.147383,-0.179946,0.248996,0.292470
2023-12-01,AVB,172.899994,177.979996,172.300003,177.940002,1174600.0,AvalonBay Communities,Real Estate,Multi-Family Residential REITs,3.705625,...,3.705625,170.863334,174.897256,0.085861,-0.009324,0.000334,0.057455,0.039141,0.411647,0.420281


In [7]:
df.columns

Index(['Symbol', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name', 'Sector',
       'Industry', 'ATR_x', 'ATR_y', 'ATR', '50D_SMA', '200D_SMA', '1M', '3M',
       '6M', '12M', 'RS IBD', 'RS Rank', 'RS Rank 20D MA'],
      dtype='object')

In [31]:
# lib.read('settings').data
lib.list_symbols()
lib.read('strategies').data

,name,filename,description,target_weight,min_weight,max_weight,params
SVIX,Short VIX Future,SVIX.py,If the Volatility Risk Premium is positive and...,0.05,0.02,0.08,"{'target_weight': 0.05, 'min_weight': 0.02, 'm..."
test1,test,test.py,,1,1,1,[Errno 2] No such file or directory: '/Users/j...


In [37]:
df = lib.read('strategies').data

In [38]:
df["active"] = str(True)

In [39]:
lib.write('strategies',df)

VersionedItem(symbol='strategies', library='general', data=n/a, version=5, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [40]:
df = lib.read('strategies').data

In [41]:
df

,name,filename,description,target_weight,min_weight,max_weight,params,active
SVIX,Short VIX Future,SVIX.py,If the Volatility Risk Premium is positive and...,0.05,0.02,0.08,"{'target_weight': 0.05, 'min_weight': 0.02, 'm...",True
test1,test,test.py,,1,1,1,[Errno 2] No such file or directory: '/Users/j...,True
